# 5.3 Lab: Cross-Validation and the Bootstrap

## 5.3.1 The Validation Set Approach

### estimate test error for linear, quadratic, cubic regressions

In [1]:
import numpy as np
import pandas as pd

pd.set_option('precision',2)
pd.set_option('display.max_rows',12)
pd.set_option('display.float_format', '{:20,.2f}'.format)

In [2]:
#데이터 불러오기

auto=pd.read_csv("C:/Users/Coby/Desktop/ESC_ML_STUDY/datasets/Auto.csv")


#na는 ?처리
auto=pd.read_csv("C:/Users/Coby/Desktop/ESC_ML_STUDY/datasets/Auto.csv", na_values=["?"])

#na drop
auto.dropna(axis=0, inplace=True) #inplace해서 auto에 바로 반영 #axis=0은 열 방향으로 drop

auto.isna().sum()
auto.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [3]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.00,8,307.00,130.00,3504,12.00,70,1,chevrolet chevelle malibu
1,15.00,8,350.00,165.00,3693,11.50,70,1,buick skylark 320
2,18.00,8,318.00,150.00,3436,11.00,70,1,plymouth satellite
3,16.00,8,304.00,150.00,3433,12.00,70,1,amc rebel sst
4,17.00,8,302.00,140.00,3449,10.50,70,1,ford torino


In [4]:
auto.cylinders=auto.cylinders.astype('category') #numeric에서 category로
auto.name=auto.name.astype('category') #str에서 category로

In [5]:
auto.reset_index(inplace=True)
auto

,index,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,0,18.00,8,307.00,130.00,3504,12.00,70,1,chevrolet chevelle malibu
1,1,15.00,8,350.00,165.00,3693,11.50,70,1,buick skylark 320
2,2,18.00,8,318.00,150.00,3436,11.00,70,1,plymouth satellite
3,3,16.00,8,304.00,150.00,3433,12.00,70,1,amc rebel sst
4,4,17.00,8,302.00,140.00,3449,10.50,70,1,ford torino
...,...,...,...,...,...,...,...,...,...,...
387,392,27.00,4,140.00,86.00,2790,15.60,82,1,ford mustang gl
388,393,44.00,4,97.00,52.00,2130,24.60,82,2,vw pickup
389,394,32.00,4,135.00,84.00,2295,11.60,82,1,dodge rampage
390,395,28.00,4,120.00,79.00,2625,18.60,82,1,ford ranger


In [6]:
auto['horsepower_2']=pow(auto.horsepower,2)
auto['horsepower_3']=np.power(auto.horsepower,3)
auto['horsepower_4']=auto.horsepower**4
auto['horsepower_5']=pow(auto.horsepower,5)

In [7]:
# Polynomial Features using sklearn:
from sklearn.preprocessing import PolynomialFeatures
pol=PolynomialFeatures(degree=5, interaction_only=False, include_bias=False)
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html
#interaction term만 추가, bias항, 즉 constant항 추가
polyf=pol.fit_transform(auto.loc[:,"horsepower"].values.reshape(-1,1)) #.values는 array로 출력 #column vector형성
#이렇게 하면 polf는 1열이 horsepower1차 2열이 2차 ... 5열이 5차인 매트릭스 형태의 array로 만들어짐.
polyf #(nx5)형태

array([[1.30000000e+02, 1.69000000e+04, 2.19700000e+06, 2.85610000e+08,
        3.71293000e+10],
       [1.65000000e+02, 2.72250000e+04, 4.49212500e+06, 7.41200625e+08,
        1.22298103e+11],
       [1.50000000e+02, 2.25000000e+04, 3.37500000e+06, 5.06250000e+08,
        7.59375000e+10],
       ...,
       [8.40000000e+01, 7.05600000e+03, 5.92704000e+05, 4.97871360e+07,
        4.18211942e+09],
       [7.90000000e+01, 6.24100000e+03, 4.93039000e+05, 3.89500810e+07,
        3.07705640e+09],
       [8.20000000e+01, 6.72400000e+03, 5.51368000e+05, 4.52121760e+07,
        3.70739843e+09]])

In [8]:
#split data

from sklearn.model_selection import train_test_split

#X행렬, y벡터 형성
X,y=auto.loc[:,['horsepower','horsepower_2','horsepower_3']], auto.mpg 

#training set, testing set으로 나눔
#https://stackoverflow.com/questions/28064634/random-state-pseudo-random-number-in-scikit-learn

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42) #random state는 seed와 비슷

In [9]:
X_train.loc[:,'horsepower'].values.shape
# .values까지만 하면 shape (196,) -> 196개 내열 -> column matrix로 바꿔야함

(196,)

In [10]:
X_train.loc[:,["horsepower"]]

,horsepower
222,130.00
368,92.00
129,65.00
38,175.00
11,160.00
...,...
71,150.00
106,100.00
270,85.00
348,65.00


In [11]:
X_train.loc[:,['horsepower','horsepower_2']]

,horsepower,horsepower_2
222,130.00,"16,900.00"
368,92.00,"8,464.00"
129,65.00,"4,225.00"
38,175.00,"30,625.00"
11,160.00,"25,600.00"
...,...,...
71,150.00,"22,500.00"
106,100.00,"10,000.00"
270,85.00,"7,225.00"
348,65.00,"4,225.00"


In [14]:
#polynomial regression and MSE

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#ols model with intercept
lm1=LinearRegression(fit_intercept=True)
lm2=LinearRegression(fit_intercept=True)
lm3=LinearRegression(fit_intercept=True)

#lm1_fit=lm1.fit(X_train.loc[:,['horsepower']],y_train)
lm1_fit=lm1.fit(X_train.loc[:,'horsepower'].values.reshape(-1,1), y_train)
lm2_fit=lm2.fit(X_train.loc[:,['horsepower','horsepower_2']],y_train)
lm3_fit=lm3.fit(X_train.loc[:,['horsepower','horsepower_2','horsepower_3']],y_train)

lm1_predict=lm1_fit.predict(X_test.loc[:,'horsepower'].values.reshape(-1,1))
lm2_predict=lm2_fit.predict(X_test.loc[:,['horsepower','horsepower_2']])
lm3_predict=lm3_fit.predict(X_test.loc[:,['horsepower','horsepower_2','horsepower_3']])

print('lm1 MSE: ', mean_squared_error(y_test, lm1_predict))
print('lm2 MSE: ', mean_squared_error(y_test, lm2_predict))
print('lm3 MSE: ', mean_squared_error(y_test, lm3_predict))


lm1 MSE:  25.5738781896844
lm2 MSE:  22.218020050032862
lm3 MSE:  22.66767543553449


## 5.3.2 Leave-One-Out Cross-Validation

In [17]:
from sklearn.model_selection import LeaveOneOut

X,y=auto.loc[:,['horsepower','horsepower_2','horsepower_3','horsepower_4','horsepower_5']], auto.mpg 

loocv=LeaveOneOut()
print(loocv.get_n_splits(X)) #즉, number of splits를 의미
print(X.count(axis=0))

392
horsepower      392
horsepower_2    392
horsepower_3    392
horsepower_4    392
horsepower_5    392
dtype: int64


In [21]:
loocv_mse=[]
lm=LinearRegression(fit_intercept=True)

for train_index, test_index in loocv.split(X): #row index로 split해줌
    X_train, X_test = X.iloc[train_index], X.iloc[test_index] #index기준으로 해당 row를 선택
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    lm1_fit=lm.fit(X_train.loc[:,["horsepower"]],y_train)
    lm1_predict=lm1_fit.predict(X_test.loc[:,["horsepower"]])
    
    loocv_mse.append(mean_squared_error(y_test, lm1_predict))

np.array(loocv_mse).mean()
                   

24.231513517929226

In [24]:
#using sklearn cross validation score
from sklearn.model_selection import cross_val_score
lm=LinearRegression(fit_intercept=True)

cval=cross_val_score(lm, auto.loc[:,["horsepower"]],
                            auto.mpg,
                            cv=len(auto), #k=n k-Fold -> LOOCV
                            n_jobs=-1,
                            scoring='neg_mean_squared_error'
                           )
-cval.mean()

#n_job: # of processors to use, -1 means use all
#scoring: A str (see model evaluation documentation) or a scorer callable object / function with signature scorer(estimator, X, y) which should return only a single value.
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

24.23151351792923

In [26]:
# Loop for 5 degree polinomial linear regressions with LOOCV

loocv_poly={}

for i in range(1,6):
    loocv_mse=[]
    for train_index, test_index in loocv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if i==1:
            X_TRAIN=X_train.iloc[:,0:1].values.reshape(-1,1)
            X_TEST=X_test.iloc[:,0:1].values.reshape(-1,1)
        
        else:
            X_TRAIN=X_train.iloc[:,0:i]
            X_TEST=X_test.iloc[:,0:i]
        
        loocv_mse.append(mean_squared_error(y_test, LinearRegression(fit_intercept=True).fit(X_TRAIN,y_train).predict(X_TEST)))
        
        loocv_poly['lm' + str(i) + " MSE"] = np.array(loocv_mse).mean()

loocv_poly

{'lm1 MSE': 24.231513517929226,
 'lm2 MSE': 19.248213124489677,
 'lm3 MSE': 19.334984064029538,
 'lm4 MSE': 19.424430310310466,
 'lm5 MSE': 19.033212804000605}

## 5.3.3 k-Fold Cross Validation

In [27]:
from sklearn.model_selection import KFold

X,y=auto.loc[:,['horsepower','horsepower_2','horsepower_3','horsepower_4','horsepower_5']], auto.mpg 

kf=KFold(n_splits=10, shuffle=True, random_state=42)

In [65]:
#시도실패..
# Loop for 5 degree polinomial linear regressions with k-Fold CV

list1=[['horsepower'],['horsepower','horsepower_2'],['horsepower','horsepower_2','horsepower_3'],['horsepower','horsepower_2',"horsepower_3","horsepower_4"],['horsepower','horsepower_2',"horsepower_3","horsepower_4","horsepower_5"]]
cval=[]

for i in list1:
    cval+=cross_val_score(lm, auto.loc[:,i],
                          auto.mpg,
                          cv=10, 
                          n_jobs=-1,
                          scoring='neg_mean_squared_error')
-cval.mean()

ValueError: operands could not be broadcast together with shapes (0,) (10,) 

In [66]:
# Loop for 5 degree polinomial linear regressions with k-Fold CV

kf_poly = {}

for i in range(1, 6):
    
    kf_mse = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if i == 1:
            X_TRAIN = X_train.iloc[:,0:1].values.reshape(-1, 1)
            X_TEST = X_test.iloc[:,0:1].values.reshape(-1, 1)
        else:
            X_TRAIN = X_train.iloc[:,0:i]
            X_TEST = X_test.iloc[:,0:i]
        
        kf_mse.append(
            mean_squared_error(
                y_test, 
                LinearRegression(fit_intercept=True)
                .fit(
                    X_TRAIN, 
                    y_train
                )
                .predict(
                    X_TEST
                )
            )
        )
        
        kf_poly['lm' + str(i) + '_MSE'] = np.array(kf_mse).mean()

kf_poly

{'lm1_MSE': 24.199808197692477,
 'lm2_MSE': 19.22863661426802,
 'lm3_MSE': 19.26626534663196,
 'lm4_MSE': 19.35109227305059,
 'lm5_MSE': 19.02323300932039}

## 5.3.4 The Bootstrap

### Estimating the Accuracy of a Statistic of Interest

In [69]:
#data 불러오기
portfolio = pd.read_csv('C:/Users/Coby/Desktop/ESC_ML_STUDY/datasets/Portfolio.csv', index_col=0)

In [70]:
#책에 있는 alpha 정의하기

def alpha_fn(data, start_index, end_index):
    X = data['X'][start_index:end_index]
    Y = data['Y'][start_index:end_index]
    return ((np.var(Y) - np.cov(X, Y)[0][1]) / (np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0][1]))

In [71]:
#estimate alpha that minimize variance
alpha_fn(portfolio, 0, 100)

0.5766511516104118

In [72]:
from sklearn.utils import resample

#random sampling of 100 obs #복원추출
portfolio_bs = resample(portfolio, replace=True, n_samples=100)

alpha_fn(portfolio_bs, 0, 100)

0.7519845482235354

In [73]:
# Bootstrap is removed from sklearn

bs_alpha = []

#1000개의 bootstrap estimate for alpha를 list로 생성
for i in range(0, 1000):
    bs_alpha.append(
        alpha_fn(resample(portfolio, replace=True, n_samples=100), 0, 100)
    )

bs_alpha = np.array(bs_alpha)

print('Bootstrapped alpha:', bs_alpha.mean())
print('SE:',  bs_alpha.std())

Bootstrapped alpha: 0.5779782261753355
SE: 0.0920736435063152


### Estimating the Accuracy of a Linear Regression Model

In [74]:
#index한 data에 대해 회귀분석 돌릴 수 있게 함수 정의
def boot_fn(data, start_index, end_index):
    m = LinearRegression(fit_intercept=True).fit(
        data['horsepower'][start_index:end_index].values.reshape(-1, 1),
        data['mpg'][start_index:end_index]
    )
    
    return m.intercept_, m.coef_

boot_fn(auto, 0, 392) #전체 데이터에 회귀분석

(39.93586102117047, array([-0.15784473]))

In [75]:
boot_fn(resample(auto, replace=True, n_samples=392), 0, 392) 
#bootstrap estimates of one random sampling

(39.667025398426084, array([-0.15199976]))

In [76]:
#1000 bootstrap estimates (intercept, coefficient)

bs_boot = {'t1': [], 't2': []}

for i in range(0, 1000):
    bs_boot['t1'].append(
        boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[0]
    )
    bs_boot['t2'].append(
        boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[1][0]
    )

t1_es = np.array(bs_boot['t1']).mean()
t1_se = np.array(bs_boot['t1']).std()
t2_es = np.array(bs_boot['t2']).mean()
t2_se = np.array(bs_boot['t2']).std()

print('t1 bs estimate & se:', t1_es, t1_se)
print('t2 bs estimate & se:', t2_es, t2_se)

t1 bs estimate & se: 39.984762851653635 0.888615006564923
t2 bs estimate & se: -0.15810710125885266 0.007429394606738342


In [77]:
#compare
import statsmodels.formula.api as sm

ols = sm.ols('mpg ~ horsepower', data=auto).fit()
ols.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,39.9359,0.717,55.660,0.000,38.525,41.347
horsepower,-0.1578,0.006,-24.489,0.000,-0.171,-0.145


In [78]:
#bootstrap of quadratic model

def boot_fn2(data, start_index, end_index):
    m = LinearRegression(fit_intercept=True).fit(
        data[['horsepower', 'horsepower_2']][start_index:end_index],
        data['mpg'][start_index:end_index]
    )
    
    return m.intercept_, m.coef_

In [79]:
bs_boot2 = {'t1': [], 't2': [], 't3': []}

for i in range(0, 1000):
    bs_boot2['t1'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[0]
    )
    bs_boot2['t2'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][0]
    )
    bs_boot2['t3'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][1]
    )


t1_es = np.array(bs_boot2['t1']).mean()
t1_se = np.array(bs_boot2['t1']).std()
t2_es = np.array(bs_boot2['t2']).mean()
t2_se = np.array(bs_boot2['t2']).std()
t3_es = np.array(bs_boot2['t3']).mean()
t3_se = np.array(bs_boot2['t3']).std()

print('t1 bs estimate & se:', t1_es, t1_se)
print('t2 bs estimate & se:', t2_es, t2_se)
print('t3 bs estimate & se:', t3_es, t3_se)

t1 bs estimate & se: 56.87278271643702 2.175273644253121
t2 bs estimate & se: -0.4663170343636877 0.03196823510301101
t3 bs estimate & se: 0.001233141833295249 0.00011859045145377825


In [80]:
#compare
import statsmodels.formula.api as sm

ols2 = sm.ols('mpg ~ horsepower + horsepower_2', data=auto).fit()
ols2.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,56.9001,1.800,31.604,0.000,53.360,60.440
horsepower,-0.4662,0.031,-14.978,0.000,-0.527,-0.405
horsepower_2,0.0012,0.000,10.080,0.000,0.001,0.001
